In [1]:
import tensorflow as tf
from tensorflow import keras

In [6]:
!pip install -q -U keras-tuner


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
import keras_tuner as kt

In [8]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [9]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [11]:
#zdefiniowanie modelu
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [13]:
#utowrzenie wystąpienie tunera i hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [15]:
#wyszukiwanie hiperparametrów
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Wyszukiwanie hiperparametrów zakończone.
Optymalna liczba neuronów warrstwy ukrytej: {best_hps.get('units')}.{best_hps.get('learning_rate')}.
          optymalna wartość learning_rate: 
          
""")

Trial 30 Complete [00h 00m 58s]
val_accuracy: 0.8615000247955322

Best val_accuracy So Far: 0.8925833106040955
Total elapsed time: 00h 12m 59s

Wyszukiwanie hiperparametrów zakończone.
Optymalna liczba neuronów warrstwy ukrytej: 480.0.001.
          optymalna wartość learning_rate: 
          



In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train,label_train,epochs=50,validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print(f'najlepsza epoka: {(best_epoch,)}')

Epoch 1/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4897 - accuracy: 0.8275 - val_loss: 0.4024 - val_accuracy: 0.8488
Epoch 2/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3660 - accuracy: 0.8671 - val_loss: 0.3867 - val_accuracy: 0.8620
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3311 - accuracy: 0.8774 - val_loss: 0.3438 - val_accuracy: 0.8762
Epoch 4/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3037 - accuracy: 0.8889 - val_loss: 0.3410 - val_accuracy: 0.8755
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2874 - accuracy: 0.8935 - val_loss: 0.3373 - val_accuracy: 0.8758
Epoch 6/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2683 - accuracy: 0.9006 - val_loss: 0.3356 - val_accuracy: 0.8792
Epoch 7/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2563 - accuracy: 0.9031 - val_loss: 0.3248 - val_accuracy:

In [ ]:
#wykonanie instancji hipermodelu - wytrenowanie go z optymalną liczabą 
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train,label_train,epochs=best_epoch, validation_split = 0.2)

In [ ]:
#ocena hipermodelu na danych testowych
eval_result = hypermodel.evaluate(img_test,label_test)
print(f'[test loss,test accuracy]: {eval_result}')